In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feature & Vector Engineering

[Change the link]

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_1_5_flash.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


## Overview

[add overview]

[add what you learned in previous notebook and link] - skip if its first


[Context of this notebook compared to overall idea]

## Getting Started

### Install Dependencies


In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform
! pip3 install --upgrade --user --quiet PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [ ]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Define project information
PROJECT_ID = "lavi-llm-experiment"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_NAME = "mlops-for-genai" # @param {type:"string"}
# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize cloud storage
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

### Import libraries


In [ ]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)
import seaborn as sns
import matplotlib.pyplot as plt
from rich.markdown import Markdown as rich_Markdown
from rich import print as rich_print
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from wordcloud import STOPWORDS, WordCloud
from collections import Counter
import PyPDF2
import io
from io import BytesIO
from google.cloud import storage
import pandas as pd
from PIL import Image
import concurrent.futures
import random, string
import asyncio
import time
from tenacity import retry, wait_random_exponential, stop_after_attempt
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import psutil
import asyncio.locks
import os
import json
import aiohttp
import uuid

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load the models

To learn more about all [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


In [ ]:
MODEL_ID_PRO = "gemini-1.5-pro-001"  # @param {type:"string"}
MODEL_ID_FLASH = "gemini-1.5-flash-001" # @param {type:"string"}

model_pro = GenerativeModel(MODEL_ID_PRO)
model_flash = GenerativeModel(MODEL_ID_FLASH)

### Add data path

In [ ]:
prototype_data = "multimodal-finanace-qa/data/unstructured/prototype/"  # @param {type:"string"}
production_data = "multimodal-finanace-qa/data/unstructured/production/"  # @param {type:"string"}
image_output_path = "multimodal-finanace-qa/data/unstructured/temp/img"  # @param {type:"string"}
embedding_input_path = "multimodal-finanace-qa/data/embeddings"  # @param {type:"string"}

## Feature Engineeing




### Building features from PDFs

extracting page wise text and images from the documents

In [ ]:
#@title Feature Extraction Helper Functions


def randomword(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))

# def upload_gcs_image_file(bucket_object,image_gcs_path,img_bytes):
#     """Uploads a file to the bucket."""
#     blob = bucket_object.blob(image_gcs_path)
#     blob.upload_from_string(img_bytes)
#     # print(f"Image file uploaded to:  {image_gcs_path}.")
def upload_gcs_image_file(bucket_object, image_gcs_path, img_bytes):
    """Uploads a file to the bucket and returns the full GCS URI."""
    blob = bucket_object.blob(image_gcs_path)
    blob.upload_from_string(img_bytes)

    # Construct the full GCS URI
    gcs_uri = f"gs://{bucket_object.name}/{image_gcs_path}"

    # print(f"Image file uploaded to: {gcs_uri}")
    return gcs_uri


def extract_image_metadata(xObject,filename,page_num, image_output_path,bucket_object, upload_image_to_gcs=False):
    image_metadata = []
    for image_number, obj in enumerate(xObject):
        if xObject[obj]['/Subtype'] == '/Image':
            image_available = True
            # Explicitly handle desired formats
            if not xObject[obj].get('/Filter') == '/FlateDecode':
              try:
                  img = Image.open(BytesIO(xObject[obj]._data))
                  # if img.format.upper() in ['JPEG', 'PNG', 'TIFF']:
                  img_bytes = io.BytesIO()
                  img.save(img_bytes, format=img.format)
                  img_bytes = img_bytes.getvalue()

                  image_name = f"{filename.replace(' ', '_').lower().split('.')[0]}_page{page_num+1}_{randomword(6)}.{img.format.lower()}"
                  # image_gcs_path = f"gs://{image_output_path}/{image_name}"
                  image_gcs_path = image_output_path+f"/{image_name}"
                  image_size = img.size
                  # print(f"Saving image to: {image_gcs_path}")
                  # if upload_image_to_gcs:
                  if upload_image_to_gcs:
                    image_gcs_path_final = upload_gcs_image_file(bucket_object, image_gcs_path, img_bytes)
                  else:
                    image_gcs_path_final = None
                  image_metadata.append({
                      'image_available': image_available,
                      'image_counter': int(image_number+1),
                      'image_gcs_path': image_gcs_path_final,
                      'image_size': image_size
                  })
                  # else:
                  #     print(f"Unsupported image format: {img.format}")
                  #     continue
              except Exception as e:
                print("Error: ", e)
                print("Error for image in file: ", filename, "and page number: ",page_num + 1,
                      " with gcs path: ", f"gs://{blob.bucket.name}/{blob.name}")
                print(f"Error: Unable to identify image format.")
                print(f"Image Subtype: {xObject[obj]['/Subtype']}")
                print(f"Image Filter: {xObject[obj].get('/Filter', 'None')}")
                print("Skipping this. Debug this. ")
                # print("Error: ", e)
                continue
    return image_metadata


def extract_pdf_data(blob, image_output_path, upload_image_to_gcs=False, bucket_object=None):
  """Extracts text and images from a PDF blob and returns metadata."""
  pdf_content = BytesIO(blob.download_as_bytes())
  try:
    pdf_reader = PyPDF2.PdfReader(pdf_content)
    pdf_data = []
    # pdf_type = "/".join(blob.name.split("/")[1:-1])
    pdf_type = blob.name.split("/")[-2]
    filename = blob.name.split("/")[-1]
    # print("filename: ", filename)

    for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      text = page.extract_text()
      image_available = False
      image_gcs_path = None
      image_size = None

      page_metadata = {
          'text_type' : pdf_type,
          'file_name': filename,
          'gcs_path': "gs://"+blob.bucket.name+"/"+blob.name,
          'page_number': page_num+1,
          'text': text if text else None,
          'image_available': False,
          'image_counter': None,
          'image_gcs_path': None,
          'image_size': None
      }

      if '/XObject' in page['/Resources']:
        xObject = page['/Resources']['/XObject'].get_object()
        image_metadata = extract_image_metadata(xObject,filename,page_num, image_output_path,bucket_object, upload_image_to_gcs)
        for metadata in image_metadata:
          page_metadata = {
              'text_type' : pdf_type,
              'file_name': filename,
              'gcs_path': "gs://"+blob.bucket.name+"/"+blob.name,
              'page_number': page_num+1,
              'text': text if text else None,
              'image_available': metadata['image_available'],
              'image_counter': int(metadata['image_counter']),
              'image_gcs_path': metadata['image_gcs_path'],
              'image_size': metadata['image_size']
          }

          pdf_data.append(page_metadata)
      else:
        pdf_data.append(page_metadata)

    return pdf_data

  except Exception as e:
    print(f"Warning: Could not read PDF file '{blob.name}' (might be encrypted or corrupted). Error: {e}")
    return []

def get_text_from_pdf(bucket_object, prefix='', image_output_path=None,upload_image_to_gcs=False,extract_tables=False):
  """Extracts text and images from PDFs in a GCS bucket, processes in parallel."""

  all_pdf_data = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for blob in bucket_object.list_blobs(prefix=prefix):
      if blob.name.lower().endswith('.pdf'):
        # print(blob.name)
        futures.append(executor.submit(extract_pdf_data, blob, image_output_path,
                                       upload_image_to_gcs, bucket_object))

    for future in concurrent.futures.as_completed(futures):
      all_pdf_data.extend(future.result())

  return pd.DataFrame(all_pdf_data)

@retry(wait=wait_random_exponential(multiplier=1, max=120),stop=stop_after_attempt(2))
async def async_generate(prompt, gcs_uri, mime_type):
    try:
        safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }
        model = GenerativeModel(
            "gemini-1.5-flash-001",
            safety_settings = safety_settings
        )
        # print("Hitting")

        response = await model.generate_content_async(
            [prompt, Part.from_uri(gcs_uri, mime_type=mime_type)],
            stream=False,
        )
        # print(len(response.text))
        return response.text
    except Exception as e:
        print("Something failed, retrying")
        print(e)
        with retry.stop_after_attempt(2) as retry_state:
            if retry_state.attempt > 2:
                return None
        raise  # Re-raise the exception for tenacity to handle

async def batch_and_profile(gcs_uris, prompt, mime_type, batch_size=2, max_concurrent=4):
    start_time = time.time()
    memory_usage = psutil.Process().memory_info().rss / 1024**2

    semaphore = asyncio.locks.Semaphore(max_concurrent)
    async def process_batch(batch):
        async with semaphore:
            return await asyncio.gather(*[async_generate(prompt, f, mime_type) for f in batch])

    batches = [gcs_uris[i:i+batch_size] for i in range(0, len(gcs_uris), batch_size)]
    get_responses = [asyncio.create_task(process_batch(batch)) for batch in batches]
    final_response_list = [item for sublist in await asyncio.gather(*get_responses) for item in sublist]

    end_time = time.time()
    elapsed_time = end_time - start_time
    final_memory_usage = psutil.Process().memory_info().rss / 1024**2

    print(f"Batch size: {batch_size}")
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    print(f"Initial memory usage: {memory_usage:.2f} MB")
    print(f"Final memory usage: {final_memory_usage:.2f} MB")

    return final_response_list

@retry(wait=wait_random_exponential(multiplier=1, max=120),stop=stop_after_attempt(2))
async def async_generate_df(session, page_text, prompt, gcs_uri):
    try:
        safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }
        model = GenerativeModel(
            "gemini-1.5-flash-001",
            safety_settings = safety_settings
        )
        final_prompt = prompt + "**page_text:** \n" + page_text
        response = await model.generate_content_async(
            [final_prompt, Part.from_uri(gcs_uri, mime_type='image/png')],
            stream=False,
        )
        return response.text
    except Exception as e:
        print("Something failed, retrying")
        print(e)
        with retry.stop_after_attempt(2) as retry_state:
            if retry_state.attempt > 2:
                return None
        raise  # Re-raise the exception for tenacity to handle


async def process_row(row, session, semaphore):
    async with semaphore:
        return await async_generate_df(session, row['text'], image_description_prompt, row['image_gcs_path'])

async def process_dataframe(df, batch_size=100, max_concurrency=10):
    semaphore = asyncio.Semaphore(max_concurrency)
    async with aiohttp.ClientSession() as session:
        # Measure initial memory usage
        initial_memory_usage = psutil.Process().memory_info().rss / 1024 / 1024

        # Start a timer
        start_time = time.time()

        print(f"Batch size: {batch_size}")
        print(f"Initial memory usage: {initial_memory_usage:.2f} MB")

        for i in range(0, len(df), batch_size):
            chunk = df[i:i+batch_size]
            tasks = [process_row(row, session, semaphore) for index, row in chunk.iterrows()]
            results = await asyncio.gather(*tasks)
            for index, result in enumerate(results):
                df.loc[chunk.index[index], 'image_description'] = result

        # Measure final memory usage
        final_memory_usage = psutil.Process().memory_info().rss / 1024 / 1024

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        print(f"Elapsed time: {elapsed_time:.2f} seconds")
        print(f"Final memory usage: {final_memory_usage:.2f} MB")

        return df

def get_gcs_uri_list(bucket,data,file_extension):
  gcs_uri_list = []
  for blob in bucket.list_blobs():
      if blob.name.startswith(data):
        if blob.name.lower().endswith(file_extension):
          gcs_path = "gs://"+"/".join(blob.id.split("/")[:-1])
          gcs_uri_list.append(gcs_path)
  return gcs_uri_list

### Building Features from PDF Files - Text

In [ ]:
%%time
pdf_metadata_flash = get_text_from_pdf(bucket, prefix=production_data,
                       image_output_path=image_output_path,
                       upload_image_to_gcs=True,
                       )

Error:  cannot identify image file <_io.BytesIO object at 0x7f5f6b826e30>
CPU times: user 2min 35s, sys: 2.35 s, total: 2min 38s
Wall time: 2min 41s


In [ ]:
pdf_metadata_flash.shape

(1549, 9)

In [ ]:
pdf_metadata_flash.head(2)

,text_type,file_name,gcs_path,page_number,text,image_available,image_counter,image_gcs_path,image_size
0,blogpost,Google announces the Coalition for Secure AI.pdf,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Additionally, CoSAI will collaborate with orga...",False,NaN,None,None
1,blogpost,"Google Gemini updates_ Flash 1.5, Gemma 2 and ...",gs://mlops-for-genai/multimodal-finanace-qa/da...,4,1.5 Pro can now follow increasingly complex an...,False,NaN,None,None


In [ ]:
pdf_metadata_flash[pdf_metadata_flash['image_available']==True].shape

(128, 9)

In [ ]:
pdf_metadata_flash[pdf_metadata_flash['image_available']==True].head(2)

,text_type,file_name,gcs_path,page_number,text,image_available,image_counter,image_gcs_path,image_size
17,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Productivity Goodput, to measure this e ciency...",True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 924)"
18,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,4,how you can measure and maximize runtime for\n...,True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 941)"


### Building Features from Image Files

In [ ]:
image_metadata_flash = pdf_metadata_flash[pdf_metadata_flash['image_available']==True]

In [ ]:
image_metadata_flash.shape

(128, 9)

In [ ]:
image_metadata_flash.head(2)

,text_type,file_name,gcs_path,page_number,text,image_available,image_counter,image_gcs_path,image_size
17,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Productivity Goodput, to measure this e ciency...",True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 924)"
18,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,4,how you can measure and maximize runtime for\n...,True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 941)"


In [ ]:
image_description_prompt = """You are a technical image analysis expert. You will be provided with various types of images extracted from documents like research papers, technical blogs, and more.
Your task is to generate concise, accurate descriptions of the images without adding any information you are not confident about.
Focus on capturing the key details, trends, or relationships depicted in the image. Use provided **page_text:** to ground the generation.

Important Guidelines:
* Prioritize accuracy:  If you are uncertain about any detail, state "Unknown" or "Not visible" instead of guessing.
* Avoid hallucinations: Do not add information that is not directly supported by the image.
* Be specific: Use precise language to describe shapes, colors, textures, and any interactions depicted.
* Consider context in Image: If the image is a screenshot or contains text, incorporate that information into your description.
* Consider context of Page: Consider the **page_text:** to understand the context of the image.
"""

image_metadata_flash_final = await process_dataframe(image_metadata_flash, batch_size=10, max_concurrency=4)

Batch size: 10
Initial memory usage: 591.84 MB


<ipython-input-6-aefdda26c6cb>:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[chunk.index[index], 'image_description'] = result


Elapsed time: 70.04 seconds
Final memory usage: 602.57 MB


In [ ]:
image_metadata_flash_final.head(2)

,text_type,file_name,gcs_path,page_number,text,image_available,image_counter,image_gcs_path,image_size,image_description
17,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Productivity Goodput, to measure this e ciency...",True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 924)",The image depicts three components of ML Produ...
18,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,4,how you can measure and maximize runtime for\n...,True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 941)",The image depicts a diagram illustrating how G...


In [ ]:
image_metadata_flash_final.shape

(128, 10)

In [ ]:
index = 7
print("********Page Text: ********")
rich_Markdown(image_metadata_flash_final.iloc[index]['text'])
print("********Image Description: ********")
rich_Markdown(image_metadata_flash_final.iloc[index]['image_description'])

********Page Text: ********


Model as a Service Endpoint The Model as a Service Endpoint is an architectural best practice that is comprised of 
three major Cloud components: ���App Hub recently launched to general availability. App Hub is a central place for 
tracking applications, services, and workloads across your Cloud projects. It maintains records of your services to
enable their discoverability & reusability, including your AI applications and models.  ���Private Service Connect 
(PSC) for secure connectivity to AI models. This allows model producers to de�ne a PSC service a�achment that model
consumers can connect to in order to access gen AI models for inference. The model producer de�nes policies on who 
can access the gen AI models. PSC also simpli�es cross network access between consumer applications and producer   
models, includingContact sales Get started for free Cloud Blog8/13/24, 8:07 PM Networking capabilities optimize    
trafﬁc for generative AI apps | Google Cloud Blog                                                                  
https://cloud.google.com/blog/products/networking/networking-capabilities-optimize-trafﬁc-for-generative-ai-apps   
6/13

********Image Description: ********


The image depicts an architectural diagram of a Model as a Service Endpoint. It shows how clients connect to the   
endpoint through an Application Serving Tier and interact with a Cloud Load Balancing service. The endpoint then   
connects to a Private Service Connect (PSC) service, which acts as a secure gateway for accessing AI models. The   
image shows two types of models: Vertex AI (130+ models) and DIY models. It also indicates the use of TPU and GPU  
for model inference. Model consumers and producers are represented as figures on the left and right sides of the   
diagram, respectively. The diagram highlights the role of App Hub as a central place for tracking and managing AI  
applications and models within the Google Cloud environment.  The flow of communication is depicted by arrows      
connecting various components. The numbers 1, 2, and 3 indicate the sequence of steps in this flow.  The diagram   
illustrates the integration of several Google Cloud services in a comprehensive model serving architecture.

### Building Features from Audio Files

In [ ]:
batch_size = 3
max_concurrent = 4
audio_description_extraction_prompt = """Transcribe and analyze the audio, identifying key topic shifts or changes in focus. Divide the audio into segments based on these transitions.
For each segment:
* **Summarize:** Briefly describe the main topic or theme of the segment.
* **Contextualize:** Explain how this topic fits into the broader conversation or narrative.
* **Analyze:** Explore the significance of this topic, the perspectives presented, and any potential biases or underlying assumptions.
* **Synthesize:** Connect this topic to other themes or ideas medntioned in the audio, highlighting relationships and overarching patterns.
Conclude with a thematic analysis of the entire audio. Identify the most prominent themes, how they are interconnected, and the overall message or purpose of the audio.
"""

gcs_uri_list_audio = get_gcs_uri_list(bucket,production_data,'.mp3')
final_response_list_audio = await batch_and_profile(gcs_uri_list_audio,
                                                    audio_description_extraction_prompt,
                                                    "audio/mpeg",
                                                    batch_size, max_concurrent)

Batch size: 3
Elapsed time: 53.03 seconds
Initial memory usage: 600.68 MB
Final memory usage: 604.98 MB


In [ ]:
audio_metadata_flash = pd.DataFrame([gcs_uri_list_audio, final_response_list_audio]).T
audio_metadata_flash.columns = ['audio_gcs', 'audio_description']
audio_metadata_flash.head(2)

,audio_gcs,audio_description
0,gs://mlops-for-genai/multimodal-finanace-qa/da...,## Alphabet's Second Quarter 2023 Earnings Con...
1,gs://mlops-for-genai/multimodal-finanace-qa/da...,## Alphabet Q3 2023 Earnings Call: A Thematic ...


In [ ]:
rich_Markdown(audio_metadata_flash.iloc[5]['audio_description'])

Alphabet's First Quarter 2024 Earnings Conference Call: A Thematic Analysis                    

This audio is a transcript of Alphabet's first quarter 2024 earnings conference call. It features several speakers,
including Sundar Pichai, Philip Schindler, and Ruth Porat, discussing the company's financial performance and      
future outlook. The call can be divided into six distinct segments:                                                

Segment 1: Introduction and Overview (0:00 - 1:22)                                                                 

 • Summary: This segment introduces the call, welcomes participants, and provides basic logistical information.    
 • Contextualize: It sets the stage for the subsequent discussion of Alphabet's financial performance and key      
   business areas.                                                                                                 
 • Analyze: The segment establishes a formal and professional tone, emphasizing the importance of the call and the 
   participation of key executives.                                                                                
 • Synthesize: This segment introduces the main themes of the call, which include financial performance, AI        
   innovation, and growth strategies across different business units.                                              

Segment 2: Sundar Pichai's AI Focus (1:22 - 3:53)                                                                  

 • Summary: Sundar Pichai highlights the company's significant growth in revenue and focuses on their strategy for 
   AI innovation, particularly in the realm of search.                                                             
 • Contextualize: This segment transitions from the formal introduction to a discussion of Alphabet's core business
   strengths and future direction.                                                                                 
 • Analyze:  Pichai emphasizes the importance of AI, particularly generative AI, in driving future growth and its  
   impact across various products and services. He also showcases the company's leadership in research,            
   infrastructure, and execution, highlighting their commitment to AI development.                                 
 • Synthesize: This segment introduces the theme of AI as a key driver of growth and a crucial focus for Alphabet. 
   It connects to the subsequent discussion of specific applications of AI in different business units.            

Segment 3: Philip Schindler on Advertising (3:53 - 7:05)                                                           

 • Summary: Philip Schindler discusses Alphabet's advertising performance, focusing on the integration of AI in    
   their ad ecosystem.                                                                                             
 • Contextualize:  This segment focuses on a specific business unit, showcasing how AI is being used to improve    
   advertising efficiency and performance.                                                                         
 • Analyze: Schindler highlights the company's progress in using AI for targeting, bidding, and creative generation
   in advertising, showcasing its effectiveness in enhancing ad performance and driving revenue. He emphasizes the 
   use of generative AI in tools like Performance Max and Demand Gen.                                              
 • Synthesize: This segment further emphasizes the theme of AI integration, illustrating its practical application 
   in generating revenue and improving customer experience. It connects to the earlier discussion of AI as a       
   strategic focus for Alphabet.                                                                                   

Segment 4: Ruth Porat on Financial Performance (7:05 - 12:45)                                                      

 • Summary: Ruth Porat discusses Alphabet's financial results for t

### Building Features from Video Files

In [ ]:
batch_size = 5
max_concurrent = 4
video_description_extraction_prompt = """Transcribe and analyze the video, intelligently segmenting it based on shifts in topic, focus, or narrative progression.
For each identified segment:
**Concise Summary**: Distill the core theme or message in 1-2 sentences.
**Thematic Context**: How does this segment contribute to the overarching narrative or argument?
**Critical Analysis**: Delve into the segment's implications, perspectives presented, and potential biases.
**Connections**: Link this segment to other parts of the video, revealing patterns and relationships.

Conclude by synthesizing the video's main themes, their interconnections, and the overarching purpose or message.
"""
gcs_uri_list_video = get_gcs_uri_list(bucket,production_data,'.mp4')

final_response_list_video = await batch_and_profile(gcs_uri_list_video,
                                                    video_description_extraction_prompt,
                                                    "video/mp4",
                                                    batch_size, max_concurrent)

Batch size: 5
Elapsed time: 66.36 seconds
Initial memory usage: 604.97 MB
Final memory usage: 605.03 MB


In [ ]:
video_metadata_flash = pd.DataFrame([gcs_uri_list_video, final_response_list_video]).T
video_metadata_flash.columns = ['video_gcs', 'video_description']
video_metadata_flash.head(2)

,video_gcs,video_description
0,gs://mlops-for-genai/multimodal-finanace-qa/da...,## Video Transcription and Analysis: Can Gemin...
1,gs://mlops-for-genai/multimodal-finanace-qa/da...,## Video Transcription and Analysis: Gemini & ...


In [ ]:
rich_Markdown(video_metadata_flash.iloc[5]['video_description'])

Google's Gemini: A Multimodal AI Model for Universal Access                            

Segment 1: 00:00-00:26                                                                                             

Concise Summary: Sundar Pichai, CEO of Google and Alphabet, introduces the video by highlighting Google's mission  
to organize the world's information and make it universally accessible. He emphasizes the increasing complexity of 
this task and the need for a breakthrough in AI technology.                                                        

Thematic Context: This segment sets the stage for the introduction of Gemini, Google's new AI model, by outlining  
the company's longstanding vision for AI's role in accessing and understanding information.                        

Critical Analysis:  Pichai presents Google's mission in a positive light, implying their commitment to             
accessibility and knowledge sharing. However, the segment lacks specifics on how Google addresses concerns         
regarding data privacy and potential biases in AI algorithms.                                                      

Connections: This segment connects with the later segments that showcase Gemini's capabilities in processing       
multiple data modalities, ultimately fulfilling the mission stated here.                                           

Segment 2: 00:27-01:06                                                                                             

Concise Summary:  Demis Hassabis, CEO of Google DeepMind, explains his belief that AI is the most consequential    
technology for humanity. He introduces Gemini, a multimodal AI model, as the first step towards a truly universal  
AI.                                                                                                                

Thematic Context: This segment further emphasizes the importance of AI and introduces Gemini as a significant      
advancement in the field. It sets the stage for a technical explanation of Gemini's unique features.               

Critical Analysis: The segment leans heavily on the positive potential of AI, but does not delve into potential    
risks associated with advanced AI technology.  The use of "consequential" technology may evoke a sense of          
inevitability and suggests AI development is a foregone conclusion.                                                

Connections: This segment connects with the following segments that delve into the technical details of Gemini's   
multimodal capabilities and its training process.                                                                  

Segment 3: 01:07-01:42                                                                                             

Concise Summary: Jeff Dean, Chief Scientist at Google DeepMind and Google Research, explains Gemini's unique       
multimodal approach.  Unlike traditional AI models, Gemini is built from the ground up to seamlessly process text, 
code, audio, images, and video.                                                                                    

Thematic Context: This segment focuses on the technical advantages of Gemini's multimodal nature, highlighting its 
ability to handle complex, multi-faceted information.                                                              

Critical Analysis: This segment showcases the technological prowess of Google and DeepMind, demonstrating their    
advanced understanding of AI. However, it lacks a discussion of the potential ethical implications of a system that
can process and understand so many different data modalities.                                                      

Connections: This segment connects to earlier segments by demonstrating how Gemini allows Google to fulfill their  
mission of making information more accessible, and it foreshadows the discussion of Gemini's ethical implications  
in later segments.                                                   

In [ ]:
# audio_metadata_flash.to_csv("audio_metadata_flash.csv")

In [ ]:
# video_metadata_flash.to_csv("video_metadata_flash.csv")

In [ ]:
audio_metadata_flash
video_metadata_flash
pdf_metadata_flash

### Splitting/Chunking the text

In [ ]:
import uuid
uuid.uuid4()

UUID('edd63637-ac10-46f5-8333-511a6bd76c70')

In [ ]:
def assign_unique_uuids(dataframes):
    """Assigns unique UUIDs to each row of multiple dataframes.

    Args:
        dataframes (list): A list of pandas DataFrames.

    Returns:
        list: A list of DataFrames with the 'uid' column added.
    """


    result_dataframes = []
    for df in dataframes:
        df['uid'] = df.apply(lambda row: str(uuid.uuid4().hex), axis=1)
        result_dataframes.append(df)

    return result_dataframes

def split_text_into_chunks(df, text_column, chunk_size):
    """Splits text into chunks of specified size, preserving other column values and adding a chunk number column."""

    # Create a list of new dataframes, one for each chunk
    new_dfs = []
    for _, row in df.iterrows():
        text_chunks = [row[text_column][i:i + chunk_size] for i in range(0, len(row[text_column]), chunk_size)]
        for chunk_index, chunk in enumerate(text_chunks):
            new_row = row.copy()  # Copy all other columns
            new_row[text_column] = chunk
            new_row['chunk_number'] = chunk_index + 1  # Add chunk number starting from 1
            new_dfs.append(pd.DataFrame([new_row]))

    return pd.concat(new_dfs, ignore_index=True)  # Combine into single dataframe

In [ ]:
%%time
# Step 2 [Why do we still do chunking? Explain ----
# 1) show the token count
# 2) reduce noise while search  ]
# latency and cost consideriation - you can still do that, but would it make sense
# Out of 5M token "information" -> you would still want to makes ure that the 1M that you send are the most relevant 1M
# Chunking the Text to smaller size to make precise match with queries

chunk_size = 500
extracted_text_chunk_df = split_text_into_chunks(pdf_metadata_flash[~pdf_metadata_flash['text'].isnull()], 'text', chunk_size)
image_metadata_chunk_df = split_text_into_chunks(image_metadata_flash_final, 'image_description', chunk_size)
video_metadata_chunk_df = split_text_into_chunks(video_metadata_flash, 'video_description', chunk_size)
audio_metadata_chunk_df = split_text_into_chunks(audio_metadata_flash, 'audio_description', chunk_size)

CPU times: user 21.1 s, sys: 307 ms, total: 21.5 s
Wall time: 25.5 s


In [ ]:
(extracted_text_chunk_df,
image_metadata_chunk_df,
 video_metadata_chunk_df,
 audio_metadata_chunk_df) = assign_unique_uuids([extracted_text_chunk_df,
                                                image_metadata_chunk_df,
                                                video_metadata_chunk_df,
                                                audio_metadata_chunk_df],
                                               )

In [ ]:
extracted_text_chunk_df.head()

,text_type,file_name,gcs_path,page_number,text,image_available,image_counter,image_gcs_path,image_size,chunk_number,uid
0,blogpost,Google announces the Coalition for Secure AI.pdf,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Additionally, CoSAI will collaborate with orga...",False,NaN,None,None,1,cd9776112b08477b8acc021be2881606
1,blogpost,Google announces the Coalition for Secure AI.pdf,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"ll to help organizations securely implement, t...",False,NaN,None,None,2,d1224dddebbe4be293fae3178afac4a0
2,blogpost,Google announces the Coalition for Secure AI.pdf,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"\nSafety & Security AI8/13/24, 8:09 PM Google...",False,NaN,None,None,3,5ed481cd1ac84cfa996555ede4dc4d4f
3,blogpost,"Google Gemini updates_ Flash 1.5, Gemma 2 and ...",gs://mlops-for-genai/multimodal-finanace-qa/da...,4,1.5 Pro can now follow increasingly complex an...,False,NaN,None,None,1,1f4375379555453882cc42f140d35146
4,blogpost,"Google Gemini updates_ Flash 1.5, Gemma 2 and ...",gs://mlops-for-genai/multimodal-finanace-qa/da...,4,.5 Pro can now reason\nacross image and audio ...,False,NaN,None,None,2,2bf74c1e678a49feb7750ba4e5ea90ca


In [ ]:
image_metadata_chunk_df.head()

,text_type,file_name,gcs_path,page_number,text,image_available,image_counter,image_gcs_path,image_size,image_description,chunk_number,uid
0,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Productivity Goodput, to measure this e ciency...",True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 924)",The image depicts three components of ML Produ...,1,5b44cb64368d44b3bc15cacc4798f113
1,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,3,"Productivity Goodput, to measure this e ciency...",True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 924)",the three components with arrows pointing upwa...,2,3d9f2ff35f4148b0b866fb4ce9326a8a
2,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,4,how you can measure and maximize runtime for\n...,True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 941)",The image depicts a diagram illustrating how G...,1,e0126b5d1a2a4fa4a133e9e37f4971d6
3,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,4,how you can measure and maximize runtime for\n...,True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 941)","g JAX, TensorFlow, PyTorch, Multislice Trainin...",2,01067d7b54c94916873dae95c0019220
4,blogpost,Goodput metric as measure of ML productivity _...,gs://mlops-for-genai/multimodal-finanace-qa/da...,4,how you can measure and maximize runtime for\n...,True,1.0,gs://mlops-for-genai/multimodal-finanace-qa/da...,"(2200, 941)",red text)\n* Runtime Goodput (yellow text)\n\n...,3,366904d8a49942619e7f3cd6e80adc89


In [ ]:
video_metadata_chunk_df.head()

,video_gcs,video_description,chunk_number,uid
0,gs://mlops-for-genai/multimodal-finanace-qa/da...,## Video Transcription and Analysis: Can Gemin...,1,29a270a2c8f44c1cb9e03e2bee5fa82a
1,gs://mlops-for-genai/multimodal-finanace-qa/da...,tical Analysis:** The video presents a positiv...,2,2ac70c63f9f348e8a9726a428cbc3b0e
2,gs://mlops-for-genai/multimodal-finanace-qa/da...,: 0:29-0:47**\n\n**Concise Summary:** The vide...,3,065aa2e4b8b641628fc7475bea962067
3,gs://mlops-for-genai/multimodal-finanace-qa/da...,d their relationships. This further reinforces...,4,81aa4c3128744b79a29e6e203bd12f43
4,gs://mlops-for-genai/multimodal-finanace-qa/da...,showcase Gemini's creativity and its ability ...,5,9287d043f25d44c49a5b88219fbf7ffa


In [ ]:
audio_metadata_chunk_df.head()

,audio_gcs,audio_description,chunk_number,uid
0,gs://mlops-for-genai/multimodal-finanace-qa/da...,## Alphabet's Second Quarter 2023 Earnings Con...,1,1f93d098a4a14896873b7f9eb694ae47
1,gs://mlops-for-genai/multimodal-finanace-qa/da...,"ilipp Schindler, and Ruth Porat. He then cover...",2,3fe4a8fe25c246b28fac239eee11d76a
2,gs://mlops-for-genai/multimodal-finanace-qa/da...,iscrepancies between the predictions made and ...,3,7e96489d466a4ea980f593ef82b9a6dc
3,gs://mlops-for-genai/multimodal-finanace-qa/da...,"tum, focusing on AI advancements and their imp...",4,dc0babb32f834454ad3d43decf0e49fd
4,gs://mlops-for-genai/multimodal-finanace-qa/da...,e company's commitment to AI and its potential...,5,c70aec6bb7f54008b10fefca108c3f74


In [ ]:
import json

def create_jsonl_file(extracted_text_chunk_df, video_metadata_chunk_df, audio_metadata_chunk_df,
                      bucket_object, jsonl_file_path):
    """
    Creates a JSONL file containing the combined text, video_description, and audio_description from the given dataframes.

    Args:
        extracted_text_chunk_df (pandas.DataFrame): The dataframe containing extracted text chunks.
        video_metadata_chunk_df (pandas.DataFrame): The dataframe containing video metadata.
        audio_metadata_chunk_df (pandas.DataFrame): The dataframe containing audio metadata.
    """

    json_data = []
    df_data = []

    for index, row in extracted_text_chunk_df.iterrows():
        json_data.append({"content": row['text']})

        df_data.append([row['uid'], "pdf_text", row['text']])

    for index, row in image_metadata_chunk_df.iterrows():
        json_data.append({"content": row['image_description']})

        df_data.append([row['uid'], "pdf_images", row['image_description']])

    for index, row in video_metadata_chunk_df.iterrows():
        json_data.append({"content": row['video_description']})

        df_data.append([row['uid'], "video_description", row['video_description']])

    for index, row in audio_metadata_chunk_df.iterrows():
        json_data.append({"content": row['audio_description']})

        df_data.append([row['uid'], "audio_description", row['audio_description']])

    # Convert the JSON data to a string
    jsonl_data = ""
    for item in json_data:
      jsonl_data += json.dumps(item) + "\n"

    # Upload the JSONL data to GCS
    blob = bucket.blob(jsonl_file_path+"/combined_data.jsonl")
    blob.upload_from_string(jsonl_data)
    print(f"File uploaded to GCS: {blob.public_url}")

    gcs_path_jsonl_data = f"gs://{bucket.name}/{blob.name}"

    return pd.DataFrame(json_data), pd.DataFrame(df_data, columns=['uid','type', 'content']), gcs_path_jsonl_data



In [ ]:
json_db_emb, index_db, gcs_path_jsonl_data = create_jsonl_file(extracted_text_chunk_df, video_metadata_chunk_df, audio_metadata_chunk_df,
                                          bucket,embedding_input_path)

File uploaded to GCS: https://storage.googleapis.com/mlops-for-genai/multimodal-finanace-qa/data/embeddings/combined_data.jsonl


In [ ]:
index_db.head()

,uid,type,content
0,cd9776112b08477b8acc021be2881606,pdf_text,"Additionally, CoSAI will collaborate with orga..."
1,d1224dddebbe4be293fae3178afac4a0,pdf_text,"ll to help organizations securely implement, t..."
2,5ed481cd1ac84cfa996555ede4dc4d4f,pdf_text,"\nSafety & Security AI8/13/24, 8:09 PM Google..."
3,1f4375379555453882cc42f140d35146,pdf_text,1.5 Pro can now follow increasingly complex an...
4,2bf74c1e678a49feb7750ba4e5ea90ca,pdf_text,.5 Pro can now reason\nacross image and audio ...


In [ ]:
index_db.shape

(10637, 3)

In [ ]:
gcs_path_jsonl_data

'gs://mlops-for-genai/multimodal-finanace-qa/data/embeddings/combined_data.jsonl'

In [ ]:
# The error message "Vertex AI Service Agent [redacted link], does not have required IAM permission(s) to the provided resource" indicates that the Vertex AI Service Agent associated with your project does not have the necessary permissions to write the output files to the specified Google Cloud Storage (GCS) bucket.

# Suggested Changes
# To resolve this issue, you need to grant the Vertex AI Service Agent the necessary permissions to access and write to the GCS bucket. Follow these steps:

# Identify the Vertex AI Service Agent: The service agent's email address is usually in the format service-<PROJECT_NUMBER>@gcp-sa-aiplatform.iam.gserviceaccount.com. You can find your project number in the Google Cloud Console.

# Grant Storage Object Creator Role: In the Google Cloud Console, navigate to the IAM & permissions page for your project. Find the Vertex AI Service Agent and grant it the "Storage Object Creator" role on the GCS bucket where you want to store the output files.

# Once you have granted the necessary permissions, try running the batch prediction job again.

# If the issue persists, double-check that the GCS bucket exists and that the Vertex AI Service Agent has the correct permissions.

In [ ]:
# https://cloud.google.com/bigquery/docs/generate-text-tutorial#grant-permissions

In [ ]:
%%time

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.preview import language_models

input_uri = (
    gcs_path_jsonl_data
)
# Format: `gs://BUCKET_NAME/DIRECTORY/` or `bq://project_name.llm_dataset`
output_uri = "gs://mlops-for-genai/multimodal-finanace-qa/data/embeddings/combined_data_output/"

textembedding_model = language_models.TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@003"
)

batch_prediction_job = textembedding_model.batch_predict(
    dataset=[input_uri],
    destination_uri_prefix=output_uri,
)
print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/761584077845/locations/us-central1/batchPredictionJobs/2732958643304529920
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/761584077845/locations/us-central1/batchPredictionJobs/2732958643304529920')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/2732958643304529920?project=761584077845
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/761584077845/locations/us-central1/batchPredictionJobs/2732958643304529920 current state:
JobState.JOB_STATE_QUEUED
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/761584077845/locations/us-central1/batchPredictionJobs/2732958643304529920 current state:
JobState.JOB_STATE

BatchPredictionJob 2024-08-18 14:55:57.505776
projects/761584077845/locations/us-central1/batchPredictionJobs/2732958643304529920
JobState.JOB_STATE_SUCCEEDED
CPU times: user 775 ms, sys: 123 ms, total: 897 ms
Wall time: 1min 30s


In [ ]:
def load_jsonl_from_gcs(bucket, file_path):
    """Loads a JSONL file from a GCS bucket and converts it into a DataFrame.

    Args:
        bucket_name (str): The name of the GCS bucket.
        file_path (str): The path to the JSONL file within the bucket.

    Returns:
        pandas.DataFrame: The DataFrame created from the JSONL data.
    """

    # storage_client = storage.Client()
    # bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)

    with blob.open('rb') as f:
        data = []
        for line in f:
            instance = json.loads(line)
            content = instance['instance']['content']
            predictions = instance['predictions'][0]['embeddings']['values']
            data.append({'content': content, 'predictions': predictions})

    df = pd.DataFrame(data)
    return df

In [ ]:
%%time
file_path = 'multimodal-finanace-qa/data/embeddings/combined_data_output/prediction-model-2024-08-18T14:55:57.998538Z/000000000000.jsonl'
embedding_df = load_jsonl_from_gcs(bucket, file_path)

CPU times: user 2min 1s, sys: 1.43 s, total: 2min 3s
Wall time: 2min 7s


In [ ]:
embedding_df.head()

,content,predictions
0,DKNNKQPNCTIGN[TGUWNVKPIHTQOCPK...,"[0.020302558317780495, 0.004051088821142912, -..."
1,\n6JGUGRCTVPGTUOC[PQVEQPVKPWGVQFQDWUKP...,"[0.023772958666086197, -0.025965938344597816, ..."
2,\nDCPFYKFVJVQYQTMGHHGEVKXGN[%WTTGPVN[V...,"[-0.0009582438506186008, -0.012345915660262108..."
3,\nUKIPKƒECPVKPHNWGPEGQXGTCNNOCVVGTUTGSWK...,"[0.012308558449149132, -0.02015942893922329, -..."
4, 1EVQDGT\n #OGPFGFCPF...,"[-0.021374214440584183, -0.0028167085256427526..."


In [ ]:
print("Size of embedding_df: ", embedding_df.shape)
print("Size of index_db: ", index_db.shape)

Size of embedding_df:  (10637, 2)
Size of index_db:  (10637, 3)


In [ ]:
# Joining embedding_df with the index_df
index_db_final = index_db.merge(embedding_df, on='content', how='left')

In [ ]:
index_db_final.head()

,uid,type,content,predictions
0,cd9776112b08477b8acc021be2881606,pdf_text,"Additionally, CoSAI will collaborate with orga...","[-0.0019601150415837765, -0.029520384967327118..."
1,d1224dddebbe4be293fae3178afac4a0,pdf_text,"ll to help organizations securely implement, t...","[0.027416100725531578, -0.02166818082332611, -..."
2,5ed481cd1ac84cfa996555ede4dc4d4f,pdf_text,"\nSafety & Security AI8/13/24, 8:09 PM Google...","[0.01608710177242756, -0.0351220928132534, -0...."
3,1f4375379555453882cc42f140d35146,pdf_text,1.5 Pro can now follow increasingly complex an...,"[-0.017015989869832993, -0.02816266193985939, ..."
4,2bf74c1e678a49feb7750ba4e5ea90ca,pdf_text,.5 Pro can now reason\nacross image and audio ...,"[0.003014838322997093, -0.018015660345554352, ..."


In [ ]:
# to test if mapping is done right.
test_index = 5000
print("*****original emb in embedding_db: *****\n", embedding_df.iloc[test_index]['predictions'][:5])
print("\n*****emb in index_db****\n", index_db_final[index_db_final['content']==embedding_df.iloc[test_index]['content']]['predictions'].values[0][:5])
print("\n*****Original content in embedding_db *****", embedding_df.iloc[test_index]['content'])
print("\n*****content in index_db*****", index_db_final[index_db_final['content']==embedding_df.iloc[test_index]['content']]['content'].values[0])

*****original emb in embedding_db: *****
 [0.01988452672958374, -0.0011216611601412296, -0.01900307461619377, 0.038870587944984436, 0.05879378691315651]

*****emb in index_db****
 [0.01988452672958374, -0.0011216611601412296, -0.01900307461619377, 0.038870587944984436, 0.05879378691315651]

*****Original content in embedding_db ***** creases in cash payments for repurchases of capital stock and net 
payments related to stock-based award activities, and a decrease in proceeds from the sale of interest in 
consolidated entities. 
Liquidity and Material Cash Requirements
We expect existing cash, cash equivalents, short-term marketable securities, cash flows from operations and 
financing activities to continue to be sufficient to fund our operating activities and cash commitments for investing 
and financing activities for at l

*****content in index_db***** creases in cash payments for repurchases of capital stock and net 
payments related to stock-based award activities, and a decrease 

In [ ]:
index_db_final.value_counts('type')

,count
type,
pdf_text,14277
audio_description,201
video_description,187
pdf_images,180


In [ ]:
# # Convert DataFrame to CSV string
# csv_string = index_db_final.to_csv()

# # Create a StringIO object to simulate a file-like object
# string_io = io.StringIO(csv_string)

In [ ]:
# # Upload the index_db_final data to GCS
# blob = bucket.blob("multimodal-finanace-qa/data/embeddings"+"/index_db_final.csv")
# blob.upload_from_string(index_db_final)
# print(f"File uploaded to GCS: {blob.public_url}")

In [ ]:
index_db_final.to_csv("index_db_final.csv", escapechar='\\')

## [Optional] Delete Resources [if not used further]

[add conclusions & Next Steps]

## Conclusion

[add conclusions & Next Steps]